In [129]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-nv97x6km
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-nv97x6km
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=148939 sha256=15768787db02dffc4326eb9dc6d5856574d59a9f5d5f38dc96fdf1ce60db0c87
  Stored in directory: /tmp/pip-ephem-wheel-cache-dcn_soem/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Found existing installation: flair 0.5
    Uninstalling flair-0.5:
      Successfully uninstalled flair-0.5


In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#data_folder = "./drive/My Drive/capstone/data/"
data_folder = "/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [94]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

In [96]:
benchmark.head()

,label,text
851,0,Construction is expected to be completed in th...
1045,-1,"U.S. fines American Airlines, Delta for long t..."
153,1,Sales for both the Department Store Division a...
532,0,Kauko-Telko 's centralized administration will...
292,0,"As part of the agreement , Aspocomp will also ..."


In [0]:

# benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', index = False, header = False)

In [0]:
#pd.read_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/dev.csv")

#### Build corpus

In [100]:
#corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
                                         train_file='train.csv', ## passing in file names manuelly when it can't auto detect
                                         dev_file='dev.csv',
                                         test_file='test.csv'
)

2020-05-26 01:50:15,892 Reading data from /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training
2020-05-26 01:50:15,894 Train: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/train.csv
2020-05-26 01:50:15,895 Dev: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/dev.csv
2020-05-26 01:50:15,897 Test: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/test.csv


#### Create word embeddings

In [101]:
#word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

word_embeddings = [BertEmbeddings()]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  This is separate from the ipykernel package so we can avoid doing imports until


#### First Stage Fine-tuning

In [102]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


2020-05-26 01:50:33,806 Computing label dictionary. Progress:


100%|██████████| 1314/1314 [00:01<00:00, 878.04it/s] 

2020-05-26 01:50:35,610 [b'0', b'-1', b'1']
2020-05-26 01:50:35,632 ----------------------------------------------------------------------------------------------------
2020-05-26 01:50:35,637 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-26 01:50:37,470 epoch 1 - iter 3/37 - loss 1.28199701 - samples/sec: 81.94
2020-05-26 01:50:50,166 epoch 1 - iter 6/37 - loss 1.41365312 - samples/sec: 85.52
2020-05-26 01:51:02,677 epoch 1 - iter 9/37 - loss 1.39422252 - samples/sec: 96.34
2020-05-26 01:51:16,442 epoch 1 - iter 12/37 - loss 1.29789497 - samples/sec: 89.96
2020-05-26 01:51:28,251 epoch 1 - iter 15/37 - loss 1.29454457 - samples/sec: 90.61
2020-05-26 01:51:42,470 epoch 1 - iter 18/37 - loss 1.28033601 - samples/sec: 85.29
2020-05-26 01:51:55,443 epoch 1 - iter 21/37 - loss 1.23133240 - samples/sec: 90.79
2020-05-26 01:52:08,022 epoch 1 - iter 24/37 - loss 1.23432668 - samples/sec: 78.13
2020-05-26 01:52:20,212 epoch 1 - iter 27/37 - loss 1.19750861 - samples/sec: 87.12
2020-05-26 01:52:32,796 epoch 1 - iter 30/37 - loss 1.17403806 - samples/sec: 93.63
2020-05-26 01:52:44,973 epoch 1 - iter 33/37 - loss 1.16789715 - samples/sec: 98.47
2020-05-26 01:52:57,717 epoch 1 - iter 36/37 - loss 1.19602260 - samples/sec: 9

{'dev_loss_history': [0.9351709485054016,
  0.7792381644248962,
  0.8116015791893005,
  0.7754347324371338,
  0.977726936340332,
  1.3314703702926636,
  0.8364132046699524,
  0.9065431952476501,
  0.9147321581840515,
  1.2106125354766846],
 'dev_score_history': [0.7052154195011338,
  0.7369614512471655,
  0.7414965986394558,
  0.746031746031746,
  0.7188208616780045,
  0.63718820861678,
  0.7414965986394558,
  0.7687074829931972,
  0.7551020408163265,
  0.7052154195011338],
 'test_score': 0.7442922374429224,
 'train_loss_history': [1.1883643076226518,
  0.9111006324355667,
  0.8130661345816947,
  0.7873718013634553,
  0.6987079857168971,
  0.7283295320497977,
  0.6145436787927473,
  0.609109215639733,
  0.5653021569187576,
  0.571110762454368]}

In [0]:
# finetune phase1 result with Bert+Flair embeddings
#MACRO_AVG: acc 0.7260273972602739 - f1-score 0.5739219483960016

In [104]:
from flair.data import Sentence
# create example sentence
sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

# predict class and print
classifier.predict(sentence)

print(sentence.labels)

[1 (0.8477)]


In [105]:
#  access the Sentence objects in each split directly
print(corpus.test[0])

Sentence: "It is also in the process of taking a seat on CapMan ' s board - a course which has the support of CapMan management ."   [− Tokens: 26  − Sentence-Labels: {'class': [0 (1.0)]}]


In [106]:
print(corpus.train[0])

Sentence: "Construction is expected to be completed in the summer of 2011 ."   [− Tokens: 12  − Sentence-Labels: {'class': [0 (1.0)]}]


In [107]:
#outputs detailed information on the dataset, each split, and the distribution of class labels.
stats = corpus.obtain_statistics()
print(stats)

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1168,
        "number_of_documents_per_class": {
            "0": 376,
            "-1": 400,
            "1": 392
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 37019,
            "min": 5,
            "max": 69,
            "avg": 31.694349315068493
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 146,
        "number_of_documents_per_class": {
            "0": 48,
            "-1": 53,
            "1": 45
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 4682,
            "min": 4,
            "max": 62,
            "avg": 32.06849315068493
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 147,
        "number_of_documents_per_class": {
            "0": 58,
            "1": 48,
            "-1": 41
        },


### Second Stage (train on hand annotated datasets)

#### Build corpus

In [146]:
#new_data_folder = './drive/My Drive/capstone/data/second_stage/'
new_data_folder = '/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/'
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
                                         train_file='GDP_train_df_oversampled.csv',
                                         dev_file = 'GDP_eva_df.csv',
                                         test_file = 'GDP_eva_df.csv'
)

2020-05-26 04:01:52,955 Reading data from /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled
2020-05-26 04:01:52,958 Train: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_train_df_oversampled.csv
2020-05-26 04:01:52,960 Dev: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df.csv
2020-05-26 04:01:52,962 Test: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df.csv


#### Second Stage fine-tuning

In [134]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-26 03:52:11,971 loading file /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/best-model.pt


In [135]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-26 03:52:12,960 ----------------------------------------------------------------------------------------------------
2020-05-26 03:52:12,966 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [0.9444580674171448,
  0.45924797654151917,
  0.37944352626800537,
  0.4544934928417206,
  0.7482798099517822,
  0.36143139004707336,
  0.24568629264831543,
  0.33372384309768677,
  0.5067477822303772,
  0.7507543563842773],
 'dev_score_history': [0.6363636363636364,
  0.7373737373737373,
  0.7777777777777778,
  0.7575757575757576,
  0.7373737373737373,
  0.8383838383838383,
  0.9191919191919192,
  0.8585858585858586,
  0.7777777777777778,
  0.8181818181818182],
 'test_score': 0.9191919191919192,
 'train_loss_history': [1.2316572785377502,
  1.1048037528991699,
  0.7458502948284149,
  0.6397366881370544,
  0.46796063184738157,
  0.627941370010376,
  0.28413652777671816,
  0.21732018291950225,
  0.24684788584709166,
  0.2870516419410706]}

In [0]:
# MACRO_AVG: acc 0.8787878787878788 - f1-score 0.8233082706766918

## after fixing oversampling to be on train-only

#MACRO_AVG: acc 0.9191919191919192 - f1-score 0.8770653907496012

In [137]:
## load the 2nd-stage finetuned model:

finetuned_classifier = TextClassifier.load(new_data_folder + 'best-model.pt')

2020-05-26 04:01:50,662 loading file /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/best-model.pt


In [138]:
# predict same sentence from above:

sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

finetuned_classifier.predict(sentence)

print(sentence.labels) ## correct

[-1 (0.5809)]


In [139]:
dir(sentence.labels)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [140]:
dir(sentence.labels[0])


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_score',
 '_value',
 'score',
 'to_dict',
 'value']

In [141]:
sentence.labels[0].value

'-1'

In [142]:
sentence.labels[0].score

0.5809285640716553

In [143]:
## error analysis

# get gold labels
#print(corpus.test[0])
print(corpus.test[0])
print(corpus.test[0].labels)



Sentence: "Trump claims victory as GDP grows at fastest pace since 2014 . President Donald Trump said the U.S. economy is on track to reach an annual growth rate of more than 3 per cent , as he celebrated a report Friday that the economy expanded in the second quarter at the fastest pace in four years ."   [− Tokens: 57  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (1.0)]


In [144]:
len(corpus.test)

33

In [0]:
gdp_test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df.csv",usecols=['title_desc_sent_1','title_desc'])

In [148]:
gdp_test_df.head()

,title_desc_sent_1,title_desc
0,1,S&P 500 just hit another record high — and eve...
1,1,Forget interest rate decisions. Loonie traders...
2,0,"With little to say on coronavirus, is Quebec's..."
3,0,Statistics are great unless they measure the w...
4,-1,"What to watch in China's GDP report: Trade, au..."


In [149]:
gdp_test_df['title_desc'].iloc[0]

'S&P 500 just hit another record high — and everything’s flashing Risk On. Optimism over trade deal with China'

In [150]:
correct = 0

for i in range(len(corpus.test)):
  print(corpus.test[i])
  

  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  sentence = Sentence(gdp_test_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  print(sentence.labels)

  #get gold label
  print(corpus.test[i].labels)

  #calculate correct guesses
  if sentence.labels[0].value == corpus.test[i].labels[0].value:
    correct += 1


Sentence: "S & P 500 just hit another record high — and everything ’s flashing Risk On . Optimism over trade deal with China"   [− Tokens: 23  − Sentence-Labels: {'class': [1 (1.0)]}]
[0 (0.5386)]
[1 (1.0)]
Sentence: "Forget interest rate decisions . Loonie traders are banking on immigration to Canada . Flood of immigrants and non-permanent residents to levels not seen in a century a main driver supporting Canada 's economic expansion"   [− Tokens: 35  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (0.8859)]
[1 (1.0)]
Sentence: "With little to say on coronavirus , is Quebec 's budget ' of the future ' out of sync with the present ? . Finance minister ‚ Äôs budget bets outbreak is more public health than economic problem"   [− Tokens: 38  − Sentence-Labels: {'class': [0 (1.0)]}]
[0 (0.9825)]
[0 (1.0)]
Sentence: "Statistics are great unless they measure the wrong things : Don Pittis . Statistics Canada looks to Big Data to size up Canadians , but are the numbers biased ?"   [− Tokens: 30  −

In [152]:
len(corpus.test)

26

In [0]:
correct #after train-only oversampling : 21/26 = 81% accuracy (Bert-only embedding)

# Error analysis

#reasonable wrongs

## tricky one -- econ 'stronger-than-expected". arguable golden label
# Sentence: "Consumer worries cloud best Canadian output gain in two years . Canada ’s economy recorded a stronger-than-expected rebound in the second quarter as exports recovered , but surprisingly weak consumption and business investment will cast doubts on the expansion ’s sustainability ."   [− Tokens: 42  − Sentence-Labels: {'class': [0 (1.0)]}]
# [1 (0.7199)]
# [0 (1.0)]

## doubt over golden label... not related to econ but related to interest rate
# Sentence: "Bank of Canada ’s Stephen Poloz gets chance today to put rate cut speculation on ice . Investors now see a strong chance of a cut by the Bank of Canada over the next 12 months"   [− Tokens: 36  − Sentence-Labels: {'class': [-1 (1.0)]}]
# [1 (0.8212)]
# [-1 (1.0)]

## also doubt over golden label. "improve only slightly" -- signals lower-than-expectation?
# Sentence: "Edmonton economy to improve only slightly in 2020 , city 's chief economist says . Capital city can expect ‚ Äòvery modest ‚ Äô growth , John Rose says"   [− Tokens: 29  − Sentence-Labels: {'class': [1 (1.0)]}]
# [0 (0.8225)]
# [1 (1.0)]

#unreasonable wrongs

## more related to stock market but 'optimism over trade deal' bodes well for economy. a tricky one? low confidence score
# Sentence: "S & P 500 just hit another record high — and everything ’s flashing Risk On . Optimism over trade deal with China"   [− Tokens: 23  − Sentence-Labels: {'class': [1 (1.0)]}]
# [0 (0.5386)]
# [1 (1.0)]

## recession, crisis are very negative words. how come?
# Sentence: "‘ Crisis like no other’ : IMF warns ‘ Great Lockdown’ recession will be worst in almost 100 years . In a further sign of pessimism , the IMF sketched out three alternative scenarios in which the virus lasted longer than expected , returned in 2021 or both"   [− Tokens: 48  − Sentence-Labels: {'class': [-1 (1.0)]}]
# [0 (0.9687)]
# [-1 (1.0)]



In [151]:
 correct #27/33 = 82% accuracy with Bert+Flair embeddings


 #before phase2 finetuning, only 66% accuracy with 1500 training examples of less GDP-specific sentiment (vs 77% acc with oversampling Kaggle dataset)

 #reasonable wrongs

## words in negative sense but meaning is neutral
# Sentence: "Why the Canadian economy seems divorced from traditional signals : Don Pittis . With housing and oil off the boil , why has n't the Canadian economy gone into free fall ?"   [− Tokens: 32  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.9096)]
# [0 (1.0)]

## not learning enough that sentiment should be Canada-specific
# Sentence: "Trump is threatening Iran with more sanctions — but what 's left to target ? . Current sanctions have already sent the country spiralling into a deep recession"   [− Tokens: 28  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.9762)]
# [0 (1.0)]

## not learning enough that sentiment should be GDP-specific
# Sentence: "If you thought August was bad , get ready for the worst month for Canadian stocks . In the past 10 years , the TSX has dropped an average 1.5 % in September"   [− Tokens: 33  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.5721)]
# [0 (1.0)]

## not that reasonable wrongs -- all predicted to the opposite extreme... problematic!

## 'stabilized' overshadowed by 'slowing' and 'weakest'? -- quite inconfident prediction
# Sentence: "China 's economy grew 6 % in fourth quarter as demand stabilized . China ’s economy stabilized last quarter after slowing to the weakest pace in almost three decades , with the first acceleration in investment since June signaling that a firmer recovery could be underway ."   [− Tokens: 47  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.5108)]
# [1 (1.0)]

## 'fastest growth' vs "trade tension"/ "threat"
# Sentence: "Oil drives Canada 's fastest economic growth spurt in a year . Canada ’s economy grew at the fastest pace in a year , further evidence of a solid expansion even as trade tensions with the U.S. remain a threat ."   [− Tokens: 41  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.6974)]
# [1 (1.0)]

## 'boom','rosy' vs 'bumpy ride' -- description balanced out but positive headline should have more weight.
# Sentence: "Mining boom to drive economic growth in territories beyond rest of Canada : report . The Conference Board of Canada says outlook rosy for Nunavut and Yukon , but N.W.T. in for a bumpy ride"   [− Tokens: 35  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.5803)]
# [1 (1.0)]

21

In [0]:

## check accuracy if only count confident predictions(condifence score > 0.7?)
confident_total = 0
confident_correct = 0

for i in range(len(corpus.test)):
  #print(corpus.test[i])
  

  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  sentence = Sentence(gdp_test_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  #print(sentence.labels)

  #get gold label
  #print(corpus.test[i].labels)

  #calculate correct guesses
  if sentence.labels[0].score > 0.7:
    confident_total += 1
    if sentence.labels[0].value == corpus.test[i].labels[0].value:
      confident_correct += 1

In [154]:
confident_correct

19

In [155]:
confident_total

23

In [156]:
confident_correct/confident_total ## higher accuracy, pick only confident predictions for good visualization! also for correlation calculation?

0.8260869565217391

In [0]:
## slightly better accuracy on train-only oversampling

In [0]:
## try XL-net